# 気候変化シナリオデータセット
農研機構が提供するメッシュ農業気象データから、気候変化シナリオデータを取得します。
- 事前にメッシュ農業気象データのユーザー登録が必要です
- メッシュ農業気象データ専用のPythonモジュール『AMD_Tools3.py』を使用します

[メッシュ農業気象データ公式サイト](https://amu.rd.naro.go.jp/wiki_open/doku.php?id=start)

### 準備

In [ ]:
import os
import AMD_Tools3 as AMD
import numpy as np
import pandas as pd

In [ ]:
###パラメータ
# 地点（つくば）
loc_lat = 36.057
loc_lon = 140.125

# 期間
cur_years = np.arange(2001, 2021)
cur_start = "{}-01-01"
cur_end   = "{}-12-31"
rcp_years = np.arange(2051, 2071)
rcp_start = "{}-01-01"
rcp_end   = "{}-12-31"

# 日最高気温、日降水量
elements = ["TMP_max", "APCP"]

# 気候変化シナリオのパラメータ
models = ['MIROC5']
scenarios = ['RCP2.6', 'RCP8.5']

## AMD用変数
lalodomain = [loc_lat, loc_lat, loc_lon, loc_lon]

### 出力ファイル
outdir = "data"
os.makedirs(outdir, exist_ok=True)
outfile1 = "tsukuba_anl_{}.csv"
outfile2 = "tsukuba_{}_{}_{}.csv"

### 過去20年分の実績データを取得
- データをまとめて20年分取ろうとすると、非常に時間がかかる
- データは1年ずつ取得した方が圧倒的に早い

In [ ]:
for year in cur_years:
    data1 = None
    timedomain1 = [cur_start.format(year), cur_end.format(year)]

    for element in elements:
        val, tim, lat, lon = AMD.GetMetData(element, timedomain1, lalodomain, isTile=True)
        if data1 is None:
            data1 = pd.DataFrame({'date':tim, element:val[:,0,0]})
        else:
            data1[element] = val[:,0,0]

    if data1 is not None:
        data1.to_csv(os.path.join(outdir, outfile1.format(year)), index=False)

### 気候変化シナリオから20年分のデータを取得
- 実績データの50年後を取得する
- データをまとめて20年分取ろうとすると、非常に時間がかかる
- データは1年ずつ取得した方が圧倒的に早い

In [ ]:
for year in rcp_years:
    timedomain2 = [rcp_start.format(year), rcp_end.format(year)]

    for model in models:
        for scenario in scenarios:        
            data2 = None
            for element in elements:
                val, tim, lat, lon = AMD.GetSceData(element, timedomain2, lalodomain, model, scenario, isTile=True)
                if data2 is None:
                    data2 = pd.DataFrame({'date':tim, element:val[:,0,0]})
                else:
                    data2[element] = val[:,0,0]

            if data2 is not None:
                data2.to_csv(os.path.join(outdir, outfile2.format(model,scenario,year)), index=False)